In [6]:
from metadata import Metadata
# import importlib
md_path = '/hybedata/Images/Robert/TBI_v2/sham_m3_slide4_2019Nov04/'
# config_file = 'seqfish_config_tbi_v4'
md = Metadata(md_path)
# config = importlib.import_module(config_file)

In [5]:
acqs = [i for i in md.image_table.acq.unique() if ('nucstain' in i)]
print(len(md.image_table[md.image_table.acq.isin(acqs)].Position.unique()))

300


In [2]:
import argparse
import functools
import json
import os
from typing import IO, Mapping, Union

import numpy as np
import pandas as pd
from skimage.io import imread
from slicedimage import ImageFormat

from starfish.core.util.argparse import FsExistsType
from starfish.experiment.builder import FetchedTile, TileFetcher, write_experiment_json
from starfish.types import Axes, Coordinates, CoordinateValue

@functools.lru_cache(maxsize=1)
def cached_read_fn(file_path):
    return imread(file_path)

# Primary Tile Class

In [ ]:
# class MERFISHTile(FetchedTile):
#     def __init__(self, file_path, r, ch, coordinates):
#         self.file_path = file_path
#         # how to index tiles from indices into multi-page tiff
#         # key is a tuple of round, chan. val is the index
#         self.map = {(0, 0): 0,
#                     (0, 1): 1,
#                     (1, 0): 3,
#                     (1, 1): 2,
#                     (2, 0): 4,
#                     (2, 1): 5,
#                     (3, 0): 6,
#                     (3, 1): 7,
#                     (4, 0): 9,
#                     (4, 1): 8,
#                     (5, 0): 10,
#                     (5, 1): 11,
#                     (6, 0): 12,
#                     (6, 1): 13,
#                     (7, 0): 15,
#                     (7, 1): 14}
#         self.r = r # Round
#         self.ch = ch # Channel
#         self._coordinates = coordinates 

#     @property
#     def shape(self) -> Mapping[Axes, int]:
#         return SHAPE

#     @property
#     def coordinates(self) -> Mapping[Union[str, Coordinates], CoordinateValue]:
#         return self._coordinates

#     def tile_data(self) -> IO:
#         return cached_read_fn(self.file_path)[self.map[(self.r, self.ch)], :, :]


In [3]:
class WollmanTile(FetchedTile):
    def __init__(self, file_path, r, ch, coordinates,shape_yx):
        self.file_path = file_path
        self.r = r # Round
        self.ch = ch # Channel
        self._coordinates = coordinates 
        self.shape_yx = shape_yx
        
    @property
    def shape(self) -> Mapping[Axes, int]:
        return {Axes.Y: self.shape_yx[0], Axes.X: self.shape_yx[1]}

    @property
    def coordinates(self) -> Mapping[Union[str, Coordinates], CoordinateValue]:
        """Stores coordinate information passed from the TileFetcher"""
        return self._coordinates
    
    @property
    def format(self) -> ImageFormat:
        """Image Format for Wollman data is TIFF"""
        return ImageFormat.TIFF
    
    def tile_data(self) -> IO:
        return cached_read_fn(self.file_path)


# Aux Tile Class

In [ ]:
# class MERFISHAuxTile(FetchedTile):
#     def __init__(self, file_path, coordinates):
#         self.file_path = file_path
#         self.dapi_index = 17
#         self._coordinates = coordinates

#     @property
#     def shape(self) -> Mapping[Axes, int]:
#         return SHAPE

#     @property
#     def coordinates(self) -> Mapping[Union[str, Coordinates], CoordinateValue]:
#         return self._coordinates

#     def tile_data(self) -> np.ndarray:
#         return cached_read_fn(self.file_path)[self.dapi_index, :, :]


In [4]:
class WollmanAuxTile(FetchedTile):
    def __init__(self, file_path, coordinates,shape_yx):
        self.file_path = file_path
        self._coordinates = coordinates

    @property
    def shape(self) -> Mapping[Axes, int]:
        return {Axes.Y: self.shape_yx[0], Axes.X: self.shape_yx[1]}

    @property
    def coordinates(self) -> Mapping[Union[str, Coordinates], CoordinateValue]:
        return self._coordinates

    def tile_data(self) -> np.ndarray:
        return cached_read_fn(self.file_path)


# primary_tile_fetcher

In [ ]:
# class MERFISHTileFetcher(TileFetcher):
#     def __init__(self, input_dir, is_dapi):
#         self.input_dir = input_dir
#         self.is_dapi = is_dapi
#         self.coordinates = self.parse_coordinates(input_dir)

#     @staticmethod # Rewrite to load and interact wit position metadata
#     def parse_coordinates(input_dir):
#         filename = os.path.join(input_dir, "stagePos.csv")
#         data = pd.read_csv(filename, names=['y_min', 'x_min'])
#         data['x_max'] = data['x_min'] + 200
#         data['y_max'] = data['y_min'] + 200
#         return data

#     def make_coordinates(self, fov):
#         return {
#             Coordinates.X: (
#                 float(self.coordinates.loc[fov, 'x_min']),
#                 float(self.coordinates.loc[fov, 'x_max'])
#             ),
#             Coordinates.Y: (
#                 float(self.coordinates.loc[fov, 'y_min']),
#                 float(self.coordinates.loc[fov, 'y_max'])
#             ),
#             Coordinates.Z: (0.0, 0.001)
#         }
#     #rewrite to interact with metadata
#     def get_tile(
#             self, fov_id: int, round_label: int, ch_label: int, zplane_label: int) -> FetchedTile:
#         filename = os.path.join(self.input_dir, 'fov_{}.tif'.format(fov_id))
#         file_path = os.path.join(self.input_dir, filename)
#         if self.is_dapi:
#             return MERFISHAuxTile(file_path, self.make_coordinates(fov_id))
#         else:
#             return MERFISHTile(file_path, round_label, ch_label, self.make_coordinates(fov_id))


In [14]:
class WollmanTileFetcher(TileFetcher):
    def __init__(self, input_dir, is_dapi, pixel_size=0.104, shape_x=2048, shape_y=2048, z_step=0.4):
        self.input_dir = input_dir
        self.is_dapi = is_dapi
        self.pixel_size = pixel_size
        self.shape_x = shape_x
        self.shape_y = shape_y
        self.z_step = z_step
        self.metadata = self.convert_metadata(input_dir)
    
    def convert_metadata(self,input_dir):
        md_table = Metadata(input_dir).image_table
        if self.is_dapi:
            acqs = [i for i in md_table.acq.unique() if ('nucstain' in i)]
        else:
            acqs = [i for i in md_table.acq.unique() if ('hybe' in i)]
        acqs.sort()
        round_labels = {acq:i for i,acq in enumerate(acqs)}
        md_table = md_table[md_table.acq.isin(acqs)]
        channels = md_table.Channel.unique()
        channels.sort()
        channel_labels = {channel:i for i,channel in enumerate(channels)}
        Zindexes = md_table.Zindex.unique()
        Zindexes.sort()
        zplane_labels = {zi:i for i,zi in enumerate(Zindexes)}
        md_table = md_table.drop(columns=[i for i in md_table.columns if not i in ['Position','acq','Channel','Zindex','XY','Z','filename']])
        md_table['fov_id'] = list(md_table.index)
        md_table['round_label'] = [round_labels[acq] for acq in md_table['acq']]
        md_table['channel_label'] = [channel_labels[channel] for channel in md_table['Channel']]
        md_table['zplane_label'] = [zplane_labels[zi] for zi in md_table['Zindex']]
        xy = np.stack(md_table['XY'])
        md_table['x_min'] = xy[:,0]
        md_table['x_max'] = xy[:,0]+self.shape_x*self.pixel_size
        md_table['y_min'] = xy[:,1]
        md_table['y_max'] = xy[:,1]+self.shape_y*self.pixel_size
        md_table['z_min'] = md_table['Z']
        md_table['z_max'] =  md_table['Z']+self.z_step
        return md_table

    def make_coordinates(self, fov):
        return {
            Coordinates.X: (
                float(self.metadata.loc[fov, 'x_min']),
                float(self.metadata.loc[fov, 'x_max'])
            ),
            Coordinates.Y: (
                float(self.metadata.loc[fov, 'y_min']),
                float(self.metadata.loc[fov, 'y_max'])
            ),
            Coordinates.Z: (
                float(self.metadata.loc[fov, 'z_min']),
                float(self.metadata.loc[fov, 'z_max'])
            )
        }
    
    #rewrite to interact with metadata
    def get_tile(
            self, fov_id: int, round_label: int, ch_label: int, zplane_label: int) -> FetchedTile:
        file_path = self.metadata.loc[fov_id,'filename']
        if self.is_dapi:
            return WollmanAuxTile(file_path, self.make_coordinates(fov_id),[self.shape_y,self.shape_x])
        else:
            return WollmanTile(file_path, round_label, ch_label, self.make_coordinates(fov_id),[self.shape_y,self.shape_x])


# Wrapper

In [ ]:
# def format_data(input_dir, output_dir):

#     input_dir = os.path.abspath(input_dir)
#     output_dir = os.path.abspath(output_dir)

#     def add_scale_factors(experiment_json_doc):
#         filename = os.path.join(input_dir, "scale_factors.json")
#         with open(filename, 'r') as f:
#             data = json.load(f)
#         experiment_json_doc['extras'] = {"scale_factors": data}
#         return experiment_json_doc
#     # Calculate from config file and metadata
#     num_fovs = 496

#     primary_image_dimensions = {
#         Axes.ROUND: 8,
#         Axes.CH: 2,
#         Axes.ZPLANE: 1,
#     }

#     aux_name_to_dimensions = {
#         'nuclei': {
#             Axes.ROUND: 1,
#             Axes.CH: 1,
#             Axes.ZPLANE: 1
#         }
#     }

#     write_experiment_json(output_dir,
#                           num_fovs,
#                           tile_format=ImageFormat.TIFF,
#                           primary_image_dimensions=primary_image_dimensions,
#                           aux_name_to_dimensions=aux_name_to_dimensions,
#                           primary_tile_fetcher=MERFISHTileFetcher(input_dir, is_dapi=False),
#                           aux_tile_fetcher={
#                               'nuclei': MERFISHTileFetcher(input_dir, is_dapi=True),
#                           },
#                           postprocess_func=add_scale_factors,
#                           default_shape=SHAPE
#                           )


In [8]:
def format_data(input_dir, output_dir,shape_yx = [2048,2048]):
    input_dir = os.path.abspath(input_dir)
    output_dir = os.path.abspath(output_dir)
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    md = Metadata(input_dir)
    acqs = [i for i in md.acqnames if ('hybe' in i)|('nucstain' in i)]
    fish_md_table = md.image_table[md.image_table.acq.isin(acqs)]
    poses = fish_md_table.Position.unique()
    rounds = np.unique([i.split('_')[0] for i in acqs if 'hybe' in i])
    channels = [i for i in fish_md_table.Channel.unique() if not 'DeepBlue' in i]
    zplanes = fish_md_table.Zindex.unique()
    # Calculate from config file and metadata
    num_fovs = len(poses)

    primary_image_dimensions = {
        Axes.ROUND: len(rounds),
        Axes.CH: len(channels),
        Axes.ZPLANE: len(zplanes),
    }

    aux_name_to_dimensions = {
        'nuclei': {
            Axes.ROUND: 1,
            Axes.CH: 1,
            Axes.ZPLANE: len(zplanes)
        }
    }

    write_experiment_json(output_dir,
                          num_fovs,
                          tile_format=ImageFormat.TIFF,
                          primary_image_dimensions=primary_image_dimensions,
                          aux_name_to_dimensions=aux_name_to_dimensions,
                          primary_tile_fetcher=WollmanTileFetcher(input_dir, is_dapi=False),
                          aux_tile_fetcher={
                              'nuclei': WollmanTileFetcher(input_dir, is_dapi=True),
                          },
                          default_shape={Axes.Y: shape_yx[0], Axes.X: shape_yx[1]}
                          )


In [15]:
input_dir = md_path
output_dir = os.path.join(md_path,'spacetx')
format_data(input_dir, output_dir)

TypeError: __init__() takes 4 positional arguments but 5 were given

# Second Attempt (5 FOVs)

In [1]:
import shutil
import os
output_dir = '/hybedata/Images/Robert/TBI_v2/sham_m3_slide4_2019Nov04/spacetx'
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

In [2]:
from metadata import Metadata
import numpy as np
import importlib
md_path = '/hybedata/Images/Robert/TBI_v2/sham_m3_slide4_2019Nov04/'
config_file = 'seqfish_config_tbi_v4'
md = Metadata(md_path)
config = importlib.import_module(config_file)

In [4]:
shape_x = 2048
shape_y = 2048
pixel_size = 0.104
z_step = 0.4
n_rounds = 9
round_labels_master = {'hybe'+str(i+1):i for i in range(n_rounds)}
round_labels_master['nucstain'] = n_rounds
channel_labels = {'DeepBlue':0,'FarRed':1,'Orange':2,'Green':3}
image_lables_master = {'hybe'+str(i+1):'primary' for i in range(n_rounds)}
image_lables_master['nucstain'] = 'nuclei'

md_table = md.image_table 
acqs = [i for i in md_table.acq.unique() if ('nucstain' in i)|('hybe' in i)]
image_labels = {acq:image_lables_master[acq.split('_')[0]] for acq in acqs}
round_labels = {acq:round_labels_master[acq.split('_')[0]] for acq in acqs}
md_table = md_table[md_table.acq.isin(acqs)]
fovs = md_table.Position.unique()
fovs.sort()
fov_labels = {fov:fov.split('Pos')[1] for fov in fovs}
md_table = md_table[md_table.Position.isin(['Pos'+str(i) for i in range(1,6)])]
Zindexes = md_table.Zindex.unique()
Zindexes.sort()
zplane_labels = {zi:i for i,zi in enumerate(Zindexes)}

md_table = md_table.drop(columns=[i for i in md_table.columns if not i in ['Position','acq','Channel','Zindex','XY','Z','filename']])
md_table['image_type'] = [image_labels[acq] for acq in md_table['acq']]
md_table['image_type'].loc[md_table[(md_table.Channel=='DeepBlue')&(md_table.image_type=='primary')].index] = 'dots'
md_table['fov'] = [fov_labels[fov] for fov in md_table['Position']] # probably just do random
md_table['round'] = [round_labels[acq] for acq in md_table['acq']]
md_table['ch'] = [channel_labels[channel] for channel in md_table['Channel']]
md_table['zplane'] = [zplane_labels[zi] for zi in md_table['Zindex']]
xy = np.stack(md_table['XY'])
md_table['xc_min'] = xy[:,0]
md_table['xc_max'] = xy[:,0]+shape_x*pixel_size
md_table['yc_min'] = xy[:,1]
md_table['yc_max'] = xy[:,1]+shape_y*pixel_size
md_table['zc_min'] = md_table['Z']
md_table['zc_max'] =  md_table['Z']+z_step
md_table['file'] = md_table['image_type'].astype(str)+'-f'+md_table['fov'].astype(str)+'-r'+md_table['round'].astype(str)+'-c'+md_table['ch'].astype(str)+'-z'+md_table['zplane'].astype(str)+'.tif'
md_table.head()

/home/zach/miniconda3/envs/starfish/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,Position,acq,XY,Z,Zindex,Channel,filename,image_type,fov,round,ch,zplane,xc_min,xc_max,yc_min,yc_max,zc_min,zc_max,file
0,Pos1,hybe4_9,"[2411.0, -3087.0]",3083.785,1,FarRed,/hybedata/Images/Robert/TBI_v2/sham_m3_slide4_...,primary,1,3,1,0,2411.0,2623.992,-3087.0,-2874.008,3083.785,3084.185,primary-f1-r3-c1-z0.tif
1,Pos1,hybe4_9,"[2411.0, -3087.0]",3083.785,1,Orange,/hybedata/Images/Robert/TBI_v2/sham_m3_slide4_...,primary,1,3,2,0,2411.0,2623.992,-3087.0,-2874.008,3083.785,3084.185,primary-f1-r3-c2-z0.tif
2,Pos1,hybe4_9,"[2411.0, -3087.0]",3083.785,1,DeepBlue,/hybedata/Images/Robert/TBI_v2/sham_m3_slide4_...,dots,1,3,0,0,2411.0,2623.992,-3087.0,-2874.008,3083.785,3084.185,dots-f1-r3-c0-z0.tif
3,Pos1,hybe4_9,"[2411.0, -3087.0]",3084.351,2,FarRed,/hybedata/Images/Robert/TBI_v2/sham_m3_slide4_...,primary,1,3,1,1,2411.0,2623.992,-3087.0,-2874.008,3084.351,3084.751,primary-f1-r3-c1-z1.tif
4,Pos1,hybe4_9,"[2411.0, -3087.0]",3084.351,2,Orange,/hybedata/Images/Robert/TBI_v2/sham_m3_slide4_...,primary,1,3,2,1,2411.0,2623.992,-3087.0,-2874.008,3084.351,3084.751,primary-f1-r3-c2-z1.tif


In [5]:
import os
output_dir = '/hybedata/Images/Robert/TBI_v2/sham_m3_slide4_2019Nov04/spacetx'
if not os.path.exists(output_dir): 
    os.mkdir(output_dir)
md_table['output_filename'] = output_dir+'/'+md_table['file'].astype(str)
md_table.to_csv(os.path.join(output_dir,'coordinates.csv'))
md_table.head()

,Position,acq,XY,Z,Zindex,Channel,filename,image_type,fov,round,ch,zplane,xc_min,xc_max,yc_min,yc_max,zc_min,zc_max,file,output_filename
0,Pos1,hybe4_9,"[2411.0, -3087.0]",3083.785,1,FarRed,/hybedata/Images/Robert/TBI_v2/sham_m3_slide4_...,primary,1,3,1,0,2411.0,2623.992,-3087.0,-2874.008,3083.785,3084.185,primary-f1-r3-c1-z0.tif,/hybedata/Images/Robert/TBI_v2/sham_m3_slide4_...
1,Pos1,hybe4_9,"[2411.0, -3087.0]",3083.785,1,Orange,/hybedata/Images/Robert/TBI_v2/sham_m3_slide4_...,primary,1,3,2,0,2411.0,2623.992,-3087.0,-2874.008,3083.785,3084.185,primary-f1-r3-c2-z0.tif,/hybedata/Images/Robert/TBI_v2/sham_m3_slide4_...
2,Pos1,hybe4_9,"[2411.0, -3087.0]",3083.785,1,DeepBlue,/hybedata/Images/Robert/TBI_v2/sham_m3_slide4_...,dots,1,3,0,0,2411.0,2623.992,-3087.0,-2874.008,3083.785,3084.185,dots-f1-r3-c0-z0.tif,/hybedata/Images/Robert/TBI_v2/sham_m3_slide4_...
3,Pos1,hybe4_9,"[2411.0, -3087.0]",3084.351,2,FarRed,/hybedata/Images/Robert/TBI_v2/sham_m3_slide4_...,primary,1,3,1,1,2411.0,2623.992,-3087.0,-2874.008,3084.351,3084.751,primary-f1-r3-c1-z1.tif,/hybedata/Images/Robert/TBI_v2/sham_m3_slide4_...
4,Pos1,hybe4_9,"[2411.0, -3087.0]",3084.351,2,Orange,/hybedata/Images/Robert/TBI_v2/sham_m3_slide4_...,primary,1,3,2,1,2411.0,2623.992,-3087.0,-2874.008,3084.351,3084.751,primary-f1-r3-c2-z1.tif,/hybedata/Images/Robert/TBI_v2/sham_m3_slide4_...


In [6]:
import shutil
from ipypb import ipb
from tqdm import tqdm
import os
output_dir = '/hybedata/Images/Robert/TBI_v2/sham_m3_slide4_2019Nov04/spacetx'
if not os.path.exists(output_dir): 
    os.mkdir(output_dir)
for i,row in ipb(md_table.iterrows(),len(md_table)):
    os.symlink(row.filename,row.output_filename)

 [████████████████████████████████████████████████████████████] 4480/4480 [00:02<00:00, 0.00s/it]

# Right way (Way too slow)

In [14]:
import os
from slicedimage import ImageFormat
from starfish.core.experiment.builder.structured_formatter import format_structured_dataset
output_dir = '/hybedata/Images/Robert/TBI_v2/sham_m3_slide4_2019Nov04/spacetx'
format_structured_dataset(output_dir,os.path.join(output_dir,'coordinates.csv'),output_dir,ImageFormat.TIFF,in_place=True)

In [15]:
from starfish import Experiment
exp = Experiment.from_json(os.path.join(output_dir,'experiment.json'))

In [16]:
print(exp.fovs())
print(repr(exp.fov().get_image('primary')))

[<starfish.FieldOfView>
  Primary Image: <slicedimage.TileSet (z: 32, c: 3, r: 9, x: 2048, y: 2048)>
  Auxiliary Images:
    nuclei: <slicedimage.TileSet (z: 32, c: 1, r: 1, x: 2048, y: 2048)>, <starfish.FieldOfView>
  Primary Image: <slicedimage.TileSet (z: 32, c: 3, r: 9, x: 2048, y: 2048)>
  Auxiliary Images:
    nuclei: <slicedimage.TileSet (z: 32, c: 1, r: 1, x: 2048, y: 2048)>, <starfish.FieldOfView>
  Primary Image: <slicedimage.TileSet (z: 32, c: 3, r: 9, x: 2048, y: 2048)>
  Auxiliary Images:
    nuclei: <slicedimage.TileSet (z: 32, c: 1, r: 1, x: 2048, y: 2048)>, <starfish.FieldOfView>
  Primary Image: <slicedimage.TileSet (z: 32, c: 3, r: 9, x: 2048, y: 2048)>
  Auxiliary Images:
    nuclei: <slicedimage.TileSet (z: 32, c: 1, r: 1, x: 2048, y: 2048)>, <starfish.FieldOfView>
  Primary Image: <slicedimage.TileSet (z: 32, c: 3, r: 9, x: 2048, y: 2048)>
  Auxiliary Images:
    nuclei: <slicedimage.TileSet (z: 32, c: 1, r: 1, x: 2048, y: 2048)>]
<starfish.ImageStack (r: 9, c: 3, 

# Hacky but way faster

# FOV Level JSON

In [7]:
from ipypb import ipb
import json
import hashlib
import os
import pandas as pd
import multiprocessing
import sys
from pathlib import Path
def sha256(path):
    with open(os.fspath(path), "rb") as fh:
        return hashlib.sha256(fh.read()).hexdigest()

def fov_wrapper(fov_table):
    output_dir = ''.join('/'+i for i in fov_table['output_filename'].iloc[0].split('/')[1:-1])
    for image_type in ['primary','nuclei']:
        fov_json(output_dir,fov_table,image_type)
        
def fov_json(output_dir,fov_table,image_type):
    fov_table = fov_table[fov_table['image_type']==image_type]
    fov = fov_table.fov.iloc[0]
    json_out = {}
    json_out['default_tile_format'] = 'TIFF'
    json_out['dimensions'] = ['y', 'z', 'c', 'zc', 'x', 'yc', 'r', 'xc']
    json_out['extras'] = {}
    json_out['shape'] = {'c': len(fov_table['ch'].unique()), 'r': len(fov_table['round'].unique()), 'z': len(fov_table['zplane'].unique())} # Input
    json_out['tiles'] = []
    json_out['version'] = '0.1.0'
    for i,row in fov_table.iterrows():
        tile_out = {}
        tile_out['coordinates'] = {'xc': [row['xc_min'], row['xc_max']],
                                 'yc': [row['yc_min'], row['yc_max']],
                                 'zc': [row['zc_min'], row['zc_max']]}
        tile_out['file'] = row['file']
        tile_out['indices'] = {'c': row['ch'], 'r': row['round'], 'z': row['zplane']}
        tile_out['sha256'] = sha256(Path(row['output_filename'])) # Input
        tile_out['tile_format'] = 'TIFF'
        tile_out['tile_shape'] = {'x': 2048, 'y': 2048}
        json_out['tiles'].append(tile_out)
    fname = image_type+'-fov_'+str(fov).zfill(3)+'.json'
    with open(os.path.join(output_dir,fname), 'w') as outfile:
        json.dump(json_out, outfile, indent=4, sort_keys=True, ensure_ascii=False)
    return None

output_dir = '/hybedata/Images/Robert/TBI_v2/sham_m3_slide4_2019Nov04/spacetx/'
md_table = pd.read_csv(os.path.join(output_dir,'coordinates.csv'),index_col=0)
Input = []
for fov in ipb(md_table.fov.unique()):
    fov_table = md_table[md_table.fov==fov]
    for image_type in ['primary','nuclei','dots']:
        fov_json(output_dir,fov_table,image_type=image_type)
        
#     Input.append(fov_table)
    
# ncpu = 25
# with multiprocessing.Pool(ncpu) as ppool:
#     sys.stdout.flush()
#     for i in ipb(ppool.imap(fov_wrapper,Input),len(Input)):
#         pass
#     ppool.close()
#     sys.stdout.flush


 [████████████████████████████████████████████████████████████] 5/5 [05:09<01:02, 61.81s/it]

# Higher level JSON

In [8]:
def image_type_json(md_table,output_dir,image_type):
    out = {}
    out['contents'] = {'fov_'+str(i).zfill(3):image_type+'-fov_'+str(i).zfill(3)+'.json' for i in md_table.fov.unique()}
    out['extras'] = None
    out['version'] = '0.1.0'
    with open(os.path.join(output_dir,image_type+'.json'),'w') as outfile:
        json.dump(out,outfile,indent=4,sort_keys=True,ensure_ascii=False)
output_dir = '/hybedata/Images/Robert/TBI_v2/sham_m3_slide4_2019Nov04/spacetx/'
for image_type in ['primary','nuclei','dots']:
    image_type_json(md_table,output_dir,image_type)

In [9]:
out = {'version':'5.0.0',
       'images':{'primary':'primary.json','nuclei':'nuclei.json','dots':'dots.json'},
       'extras':{},
       'codebook':'codebook.json'}
with open(os.path.join(output_dir,'experiment.json'),'w') as outfile:
    json.dump(out,outfile,indent=4)
    

In [10]:
import importlib
config_file = 'seqfish_config_tbi_v4'
config = importlib.import_module(config_file)

from starfish.core.codebook.codebook import Codebook
n_rounds = 9
round_labels_master = {'hybe'+str(i+1):i for i in range(n_rounds)}
round_labels_master['nucstain'] = n_rounds
channel_labels = {'DeepBlue':0,'FarRed':1,'Orange':2,'Green':3}

codebook_array = []
for i,row in config.codewords.iterrows():
    codeword = []
    for bit,[fluorophore,hybe,channel] in enumerate(config.bitmap):
        r = round_labels_master[hybe]
        c = channel_labels[channel]
        v = row.barcode[bit]
        codeword.append({"r":r,"c":c,"v":v})
    codebook_array.append({"codeword":codeword,"target":row['name']})
codebook = Codebook.from_code_array(codebook_array)
codebook_json_filename = "codebook.json"
codebook.to_json(os.path.join(output_dir, codebook_json_filename))

In [11]:
from starfish import Experiment
exp = Experiment.from_json(os.path.join(output_dir,'experiment.json'))

In [12]:
print(exp.fovs())
print(repr(exp.fov().get_image('primary')))

[<starfish.FieldOfView>
  Primary Image: <slicedimage.TileSet (c: 2, z: 32, r: 9, x: 2048, y: 2048)>
  Auxiliary Images:
    nuclei: <slicedimage.TileSet (c: 1, z: 32, r: 1, x: 2048, y: 2048)>
    dots: <slicedimage.TileSet (c: 1, z: 32, r: 9, x: 2048, y: 2048)>, <starfish.FieldOfView>
  Primary Image: <slicedimage.TileSet (c: 2, z: 32, r: 9, x: 2048, y: 2048)>
  Auxiliary Images:
    nuclei: <slicedimage.TileSet (c: 1, z: 32, r: 1, x: 2048, y: 2048)>
    dots: <slicedimage.TileSet (c: 1, z: 32, r: 9, x: 2048, y: 2048)>, <starfish.FieldOfView>
  Primary Image: <slicedimage.TileSet (c: 2, z: 32, r: 9, x: 2048, y: 2048)>
  Auxiliary Images:
    nuclei: <slicedimage.TileSet (c: 1, z: 32, r: 1, x: 2048, y: 2048)>
    dots: <slicedimage.TileSet (c: 1, z: 32, r: 9, x: 2048, y: 2048)>, <starfish.FieldOfView>
  Primary Image: <slicedimage.TileSet (c: 2, z: 32, r: 9, x: 2048, y: 2048)>
  Auxiliary Images:
    nuclei: <slicedimage.TileSet (c: 1, z: 32, r: 1, x: 2048, y: 2048)>
    dots: <slicedi